In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
model_input=[0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
model_output=[3]
train_data='seoul_housing_integrated.csv'

In [4]:
train_dataX = pd.read_csv(train_data, usecols = model_input)
train_dataY = pd.read_csv('temp.csv')

In [6]:
train_dataX.head(5)


,법정동코드,전용면적(㎡),계약년월,층,건축년도,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
0,1117010100,83.04,201405,4,2012,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
1,1117010100,40.99,201411,1,1973,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
2,1117010100,68.61,201407,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
3,1117010100,76.27,201501,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
4,1117010100,76.27,201503,1,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0


In [5]:
code_list = train_dataX['법정동코드']
code_list = code_list.drop_duplicates()
code = {}
cnt = 0 
for co in code_list:
    code[co]=cnt
    cnt = cnt + 1
    
X = train_dataX

In [6]:
X['나이'] = X['계약년월']/100 - X['건축년도']
X['나이'] = X['나이'].astype(int)

In [7]:
X = X.drop(columns=['계약년월', '건축년도'], axis=1)

In [8]:
idx = 0
for c in X['법정동코드']:
    try :
        #print(c)
        X.at[idx, '법정동코드'] = code[c]
        #print(code[c])
        idx = idx + 1
    except :
        continue

In [9]:
X.head(1)

,법정동코드,전용면적(㎡),층,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원,나이
0,0,83.04,4,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,2


In [10]:
train_dataY['price'].head(5)

0    45700
1    16000
2    42000
3    49800
4    50000
Name: price, dtype: int64

In [11]:
Y = train_dataY['price']
Y = Y/1000
Y = Y.astype(int)

In [12]:
XX = X.drop(columns=['법정동코드'])

In [13]:
X_train, X_test , y_train, y_test = train_test_split(XX, Y, test_size = 0.2, random_state=123456)

In [14]:
np.random.seed(123456)

model = Sequential()

model.add(Dense(40, input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

model.compile(loss='mean_absolute_error', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [48]:
model.fit(X_train, y_train, batch_size=2048, epochs=300, verbose=1)

Epoch 1/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0921
Epoch 2/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0683
Epoch 3/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0764
Epoch 4/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0726
Epoch 5/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.1081
Epoch 6/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.1073
Epoch 7/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0715
Epoch 8/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0428
Epoch 9/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0481
Epoch 10/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0725
Epoch 11/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.0753
Epoch 12/300
219070

KeyboardInterrupt: 

In [59]:
X_sample=pd.read_csv('sample.csv')

In [62]:
y_sample=pd.DataFrame(data=[840, 99], columns=['price'])

In [65]:
y_sample.head(2)

,price
0,840
1,99


In [66]:
sy_pred=model.predict(X_sample)
print(y_sample-sy_pred)

        price
0  404.407928
1    4.197006


In [53]:
ty_pred=model.predict(X_test)
print(y_test)
#resi=y_test-ty_pred
#print(resi)
#resi.to_csv('resi.csv')

121377     80
89345      51
225145     99
28080      40
230205     83
186282    144
1765       30
129214     33
156370     40
262431     65
56145      23
124653     41
255602     53
237845     54
129919     40
45226      14
114297     89
195543    175
242570     81
176175     70
236142     66
149569     59
135890     44
234672     66
42847      32
245101     29
244110     91
259113     38
115827     39
221326    115
         ... 
146020    107
153236     18
247709     37
19725      65
28083      32
117791     55
81283      39
132435     49
247161     62
57525      83
65169      42
227406     36
253588     73
187279     82
30843      49
149194     35
266902     39
89219      16
264626     44
181792     34
46777      41
86344      38
36778      65
32413      84
241920     89
130806     50
105248     38
125721     31
259306     38
236637     55
Name: price, Length: 54768, dtype: int64


In [54]:
print(ty_pred)

[[61.222546]
 [47.34512 ]
 [94.803   ]
 ...
 [40.293453]
 [33.07352 ]
 [59.47951 ]]


In [58]:
X_test.head(3)

,전용면적(㎡),층,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원,나이
121377,119.47,9,0.0,0.0,0.0,1.0,0.0,1.0,10.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,14
89345,84.87,11,0.0,0.2,2.0,1.0,0.0,2.6,7.6,0.4,0.0,0.8,0.0,2.6,0.0,0.2,32
225145,84.79,30,2.5,0.5,0.5,0.5,0.5,2.0,10.0,1.0,2.0,1.5,0.0,1.5,0.0,0.0,8


In [15]:
dataY_predict = model.predict(X_test)

In [29]:
labels = dataY_predict.astype(int)

In [35]:
df = pd.DataFrame(dataY_predict.astype(int))
df = df.drop_duplicates()

In [41]:
df2 = y_test.drop_duplicates()
df.append(df2)
df2=df2.drop_duplicates()

In [43]:
labels = np.array(df2)

In [44]:
import sklearn.metrics as sm
print(sm.mean_absolute_error(dataY_predict,y_test))
print(sm.mean_squared_error(dataY_predict,y_test))
print(sm.confusion_matrix(y_test, dataY_predict.astype(int),labels=labels))

11.429209335337374
379.48138627846043
[[10  1  3 ...  0  0  0]
 [ 5 34  0 ...  0  0  0]
 [ 2  0 10 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [16]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.utils.multiclass import unique_labels

font_path = '/Library/Fonts/NanumBarunGothic.otf'
font_name=fm.FontProperties(fname=font_path, size=12).get_name()
plt.rc('font', family=font_name)



%matplotlib inline

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [19]:

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
seoul_rf = RandomForestClassifier(n_estimators=50, criterion='entropy', oob_score = True, random_state=123456)
seoul_rf.fit(X_train, y_train)

predicted = seoul_rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'Out-of-bag score estimate: {seoul_rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

print("특성:", X_train.columns)
print("특성 중요도:\n{}".format(seoul_rf.feature_importances_))


Out-of-bag score estimate: 0.134
Mean accuracy score: 0.136
특성: Index(['전용면적(㎡)', '층', '랜드마크', '영화관', '쇼핑센터', '대형마트', '백화점', '지하철', '차선수',
       '순환도로', '한강', '공원', '산', '학교', '교육특구', '대학병원', '나이'],
      dtype='object')
특성 중요도:
[0.3018715  0.26344651 0.00695897 0.00637745 0.0142541  0.00969505
 0.00547708 0.02206651 0.0206721  0.00921392 0.01834082 0.01570858
 0.00805679 0.0180389  0.01160116 0.00550622 0.26271433]


In [19]:
import xgboost as xgb


In [21]:
seoul_xgb = xgb.XGBClassifier(max_depth=3,
                              learning_rate=0.02,
                              n_estimators=30,
                              gamma=0.1,
                              booster='gbtree')
seoul_xgb.fit(X_train, y_train)
#    y_pred = movie_xgb.predict(y_train)
print("훈련 세트 정확도: {:.3f}".format(seoul_xgb.score(X_train, y_train)))
# y_pred = movie_xgb.predict(y_test)
print("테스트 세트 정확도: {:.3f}".format(seoul_xgb.score(X_test, y_test)))

KeyboardInterrupt: 